In [1]:
%matplotlib inline
import glob
import pandas as pd

In [5]:
d = pd.read_excel('cius_data/cius2012datatables/Table_77_Full_time_Law_Enforcement_Employess_by_State_2012.xls')

In [125]:
FILE_MAPPING = {
    '2000': 'cius_data/2000/table2-5_vicage00.xls', # https://ucr.fbi.gov/crime-in-the-u.s/2000
    '2001': 'cius_data/2001/table2-5_vicage01.xls', # https://ucr.fbi.gov/crime-in-the-u.s/2001
    '2002': 'cius_data/2002/table2-5_vicage02.xls', # https://ucr.fbi.gov/crime-in-the-u.s/2002
    '2003': 'cius_data/2003/table2-4_age03.xls', # https://ucr.fbi.gov/crime-in-the-u.s/2003/
    '2004': 'cius_data/2004/04tbl77a.xls', # https://www2.fbi.gov/ucr/cius_04/law_enforcement_personnel/table_77.html
    '2005': 'cius_data/2005/05tbl77.xls',
    '2006': 'cius_data/CIUS2006datatables/06tbl77.xls',
    '2007': 'cius_data/CIUS2007datatables/07tbl77.xls',
    '2008': 'cius_data/CIUS2008datatables/08tbl77.xls',
    '2009': 'cius_data/CIUS2009datatables/09tbl77.xls',
    '2010': 'cius_data/CIUS2010datatables/10tbl77.xls',
    '2011': 'cius_data/CIUS2011datatables/Table_77_Full-time_Law_Enforcement_Employees_by_State_2011.xls',
    '2012': 'cius_data/cius2012datatables/Table_77_Full_time_Law_Enforcement_Employess_by_State_2012.xls',
    '2013': 'cius_data/cius2013datatables/Table_77_Full_time_Law_Enforcement_Employess_by_State_2013.xls'
}

YEARS = ['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013']

def count_by_year(year):
    file_path = FILE_MAPPING[year]
    raw = pd.read_excel(file_path)
    raw = raw[['Table 77', 'Unnamed: 7', 'Unnamed: 6', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']]
    raw.columns = ['State', 'Population', 'Number of agencies', 'Total law enforcement employees', 'Total male officers', 'Total female officers', 'Total male civilians', 'Total female civilians']
    raw['Year'] = year
    data = raw.iloc[4:, :]
    data.set_index('Year', inplace=True)
    return data[data['State'].notnull()]

years = pd.concat([count_by_year(y) for y in YEARS])
years['State'] = years['State'].apply(lambda x: x.strip())

In [127]:
dest_path = 'cius_data/processed/table-77-fte-law-enforcement/%s.csv'
years.to_csv(dest_path % 'police-employee-counts-by-state-2005-2013')

In [130]:
ny = years[years['State'] == 'NEW YORK']

In [131]:
ny

,State,Population,Number of agencies,Total law enforcement employees,Total male officers,Total female officers,Total male civilians,Total female civilians
Year,,,,,,,,
2005,NEW YORK,17905562,409,83625,52544,8351,7380,15350
2006,NEW YORK,18719867,402,86053,53597,8579,7820,16057
2007,NEW YORK,18883746,427,86952,53742,8579,8019,16612
2008,NEW YORK,18103583,342,80949,52722,8607,5780,13840
2009,NEW YORK,19120958,436,83941,53588,8572,7711,14070
